# Relationle

### (Working Title)

---

Relationle is kind of like reverse-Semantle. Instead of guessing a word based on other similar words, you are given the base word from the beginning. Your goal is to craft a unique definition for your word, based on a few types of relationships to other words.

Guesses will be scored based on how many clauses they use, and how many words they capture: the goal is to use as few clauses as possible to capture exactly one target word.

In [63]:
import requests
import time
from random import choice, shuffle
import re

In [4]:
def conceptnet(term: str, lang:str='en', all_results=True, verbose=False) -> dict:
    if verbose:
        print(f'Querying term "{term}"')
        start = time.time()
    res = requests.get(f'http://api.conceptnet.io/c/{lang}/{term}')
    if verbose:
        elapsed = time.time() - start
        print(f'Response received after {elapsed}s.')
    return res.json()

def conceptnet_query(query, lang='en', verbose=False):
    if verbose:
        print(f'Querying {query}')
        start = time.time()
    res = requests.get(query)
    if verbose:
        elapsed = time.time() - start
        print(f'Response received after {elapsed}s.')
    return res.json()

In [35]:
def labels(res, as_tuple=True):
    output = []
    edges = res['edges']
    for edge in edges:
        start = edge['start']['label']
        end   = edge['end']['label']
        rel   = edge['rel']['label']
        
        if as_tuple:
            output.append((start, rel, end))
        else:
            output.append(f'{start} {rel} {end}')
    return output

In [34]:
class QueryBuilder:
    def __init__(self):
        self.params = []

    def start(self, uri):
        self.params.append(('start', uri))

    def end(self, uri):
        self.params.append(('end', uri))

    def rel(self, relation):
        self.params.append(('rel', relation))

    def node(self, uri):
        self.params.append(('node', uri))

    def other(self, uri):
        self.params.append(('other', uri))

    def sources(self, uri):
        self.params.append(('sources', uri))

    def clear(self):
        self.params = []
    
    def _format_param(self, param, value, lang='en'):
        if param == 'rel':
            return f'rel=/r/{value}'
        elif param == 'source':
            return f'source=/s/contributor/{value}'
        else:
            return f'{param}=/c/{lang}/{value}'
            
    def query(self, lang='en'):
        base_url = 'http://api.conceptnet.io/query?'
        query_params = '&'.join([self._format_param(p, v, lang) for p, v in self.params])
        return base_url + query_params

In [67]:
def normalize(word):
    word = word.lower()
    word = re.sub('^an? ', '', word)
    return word

In [79]:
class Definition:
    def __init__(self, word):
        self.word = word
        # Clauses where the start is known
        self.clauses=[]
        # Clauses where the end is known
        self.end_clauses = []
    
    def add_clause(self, query, direction):
        assert self.word not in query
        self.clauses.append((query, direction))
    
    def evaluate(self):
        clause_sets = []

        for q, d in self.clauses:
            edges = labels(conceptnet_query(q))
            target_idx = 0 if d == 'end' else 2
            targets = set([normalize(e[target_idx]) for e in edges])
            clause_sets.append(targets)

        words = set.intersection(*clause_sets)

        if self.word in words and len(words == 1):
            print('You win!')
        return words
        

In [83]:
d = Definition('page')
q = QueryBuilder()

# q.end('test')
# q.rel('IsA')
# c1 = q.query()
# q.clear()

q.start('midterm')
q.rel('IsA')
c2 = q.query()

# d.add_clause(c1, 'end')
d.add_clause(c2, 'start')

d.evaluate()

{'exam', 'exam in a school', 'exam not a holiday', 'point'}